### 2. Autograd в PyTorch

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

При обучении нейронных сетей мы минимизируем значение функции ошибки на обучающей выборке, меняя значения параметров модели. Чтобы понять, куда нужно сместить значения параметров, нужно уметь считать градиент — именно для автоматизации этих расчётов нам и нужен Pytorch. Посмотрим, как именно он это делает.

Но перед этим вспомним правило дифференцирования сложной функции:

$
\begin{align}
\frac{\partial f}{\partial x} =
\frac{\partial f}{\partial u} \cdot 
\frac{\partial u}{\partial x}
\end{align}
$

Пример:

$$
f(x) = \sin(\ln x) \quad
u(x) = \ln x  \quad
f(u) = \sin(u)$$

$$
\frac{\partial f}{\partial x} =
\frac{\partial f}{\partial u} \cdot 
\frac{\partial u}{\partial x} = 
\cos(u) \cdot 
\frac{1}{x} = 
\cos(\ln x) \cdot 
\frac{1}{x}
$$

#### 2.1. Дифференцирование и вычислительный граф

Рассмотрим выражение $f(x, y) = x^2 + xy + (x + y)^2$ и построим его граф:

<img height=500 src="../assets/forward_pass.png">

Производная по переменной $x$:

$$
\begin{align*}
\frac{\partial f}{\partial x} &=
\color{violet}
\frac{\partial f}{\partial d} \cdot 
\frac{\partial d}{\partial a} \cdot 
\frac{\partial a}{\partial x}
\color{white}
+
\color{green}
\frac{\partial f}{\partial d} \cdot 
\frac{\partial d}{\partial b} \cdot 
\frac{\partial b}{\partial x}
\color{white}
+
\color{cyan}
\frac{\partial f}{\partial e} \cdot 
\frac{\partial e}{\partial c} \cdot 
\frac{\partial c}{\partial x} \\
&=
\color{violet}
1 \cdot 1 \cdot 2x
\color{white}
+
\color{green}
1 \cdot 1 \cdot y
\color{white}
+
\color{cyan}
1 \cdot 2c \cdot 1
\color{white}
=
2x + y + 2(x+y)
\end{align*}
$$

Для вычисления производных Pytorch строит вычислительный граф, проход по которому позволяет рассчитать градиенты по правилу производной сложной функции (chain rule).

Прямой проход:
- расчёт значения выходного тензора
- построение графа и сохранение нужных для обратного прохода данных для каждой операции

Обратный проход (вызов `.backward()` у корня графа):
- расчёт градиентов и их накопление в артибуте `.grad` каждого тензора
- распространение вычислений далее до листьев графа

Запишем выражение для $f(x, y)$, задав начальные условия $x = 2.0, y = 2.0$.

In [1]:
import torch

In [2]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
a = x**2
b = x * y
c = x + y
d = a + b
e = c**2
f = d + e
f

tensor(24., grad_fn=<AddBackward0>)

`grad_fn` означает, что `f` не просто отдельный тензор, а связан с вычислительным графом и соответствует операции `Add`

Запустим backprop и убедимся, что градиенты рассчитаны правильно:

$\frac{\partial f}{\partial x} = 2x + y + 2(x + y)$

$\frac{\partial f}{\partial y} = x + 2(x + y)$

In [3]:
f.backward()
print(x.grad)
print(y.grad)

tensor(14.)
tensor(10.)


Повторный вызов `backward()` приведёт к ошибке, потому что после предыдущего вызова граф уже уничтожен для высвобождения ресурсов. Такое поведение по умолчанию оправдано, но если мы по какой-то причине хотим сохранить граф, следует указать `.backward(retain_graph=True)`

In [4]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

f = x**2 + x * y + (x + y)**2

# вызовем backward дважды и посмотрим на градиенты:
print("Первый вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)

print("\nВторой вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)

Первый вызов
Производная по x:  tensor(14.)
Производная по x:  tensor(10.)

Второй вызов
Производная по x:  tensor(28.)
Производная по x:  tensor(20.)


После второго вызова градиенты удвоились, но здесь нет ошибки: градиенты накапливаются в поле `.grad`, и если мы хотим избавиться от истории прошлых вычислений, это стоит сделать явно

In [5]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
a = x**2
b = x * y
c = x + y
d = a + b
e = c**2
f = d + e

# вызовем backward дважды и посмотрим на градиенты:
print("Первый вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)
# обнулим градиенты, можно сделать двумя способами
x.grad = None
y.grad = torch.tensor(0.0)

print("\nВторой вызов")
f.backward(retain_graph=True)
print("Производная по x: ", x.grad)
print("Производная по x: ", y.grad)

Первый вызов
Производная по x:  tensor(14.)
Производная по x:  tensor(10.)

Второй вызов
Производная по x:  tensor(14.)
Производная по x:  tensor(10.)
